In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hii'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-calc

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

alias | wc -l

0


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1210)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-1210

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-calc/test-1210


In [6]:
# Count aliases proper
alias | wc -l

1


In [7]:
# Count functions
typeset -f | egrep '^\w+' | wc -l

0


In [80]:
## THE CORE OF CALCULATOR COMMAND FUNCTIONS

function old-calc () { echo "$@" | bc -l; }
# EX: perl-calc "2 / 4" => 0.500

## REPLACED perl- wth perl (BEFORE perlcalc.perl) DUE TO COMMAND NOT FOUND ERROR
function perl-calc () { perl perlcalc.perl -args "$@"; }
# TODO: read up on variable expansion in function environments
function perl-calc-init () { initexpr="$1"; shift; echo "$@" | perl perlcalc.perl -init="$initexpr" -; }
alias calc='perl-calc'
alias calc-prec6='perl-calc -precision=6'
alias calc-init='perl-calc-init'
alias calc-int='perl-calc -integer'
# TODO: use '$@' for '$*' (or note why not appropriate)
function old-perl-calc () { perl -e "print $*;"; }
function hex2dec { perl -e "printf '%d', 0x$1;" -e 'print "\n";'; }
function dec2hex { perl -e "printf '%x', $1;" -e 'print "\n";'; }
function bin2dec { perl -e "printf '%d', 0b$1;" -e 'print "\n";'; }
function dec2bin { perl -e "printf '%b', $1;" -e 'print "\n";'; }

alias hv='hexview.perl'

In [42]:
## TEST 1 - old-calc SEEMS TO WORK PRETTY WELL
old-calc "24 / 4.0 - (35 / 7.0) * 5" # returns -19 as float
linebr
old-calc "(2^3)*(2^2)" # returns 32
linebr
old-calc "5*4*3*2*1+1" # returns 121

-19.00000000000000000000
--------------------------------------------------------------------------------
32
--------------------------------------------------------------------------------
121


In [51]:
## TEST 2 - old-perl-calc WORKS TOO
old-perl-calc "70 + 69 + 68"
printf "\n"
linebr
old-perl-calc "8/8/8/8/8"
printf "\n"
linebr

207
--------------------------------------------------------------------------------
0.001953125
--------------------------------------------------------------------------------


In [61]:
## TEST 3A - hex2dec
hex2dec "42Df5D144" #cases don't matter in the number
linebr
hex2dec "FG" #throws an error as G is not hex
linebr

17950953796
--------------------------------------------------------------------------------
Bareword found where operator expected at -e line 1, near "0xFG"
	(Missing operator before G?)
syntax error at -e line 1, near "0xFG"
Execution of -e aborted due to compilation errors.
--------------------------------------------------------------------------------


In [65]:
## TEST 3B - hex2dec
dec2hex "128"
#inclusion of non-decimal numbers yield 0 as a result
dec2hex "A12" 

80
0


In [67]:
## TEST 3C - bin2dec
bin2dec "11110011"
linebr
#inclusion of non-binary numbers leads to error
bin2dec "10011002" 

243
--------------------------------------------------------------------------------
Illegal binary digit '2' at -e line 1, at end of line
Execution of -e aborted due to compilation errors.


: 255

In [69]:
## TEST 3D - dec2bin
dec2bin "70419"
#inclusion of non-decimal number leads to error
linebr
dec2bin "991E"

10001001100010011
--------------------------------------------------------------------------------
Bareword found where operator expected at -e line 1, near "991E"
	(Missing operator before E?)
syntax error at -e line 1, near "991E"
Execution of -e aborted due to compilation errors.


: 255

In [74]:
## TEST 4 - hv
# creates hexview of ping.sh file
hv "ping.sh"

00000000  23 21 20 2F 62 69 6E 2F - 63 73 68 20 2D 66 0A 23  #! /bin/csh -f.#
00000010  0A 23 20 70 69 6E 67 2E - 73 68 3A 20 73 69 6D 70  .# ping.sh: simp
00000020  6C 65 20 77 72 61 70 70 - 65 72 20 61 72 6F 75 6E  le wrapper aroun
00000030  64 20 74 68 65 20 70 69 - 6E 67 20 63 6F 6D 6D 61  d the ping comma
00000040  6E 64 20 72 65 64 69 72 - 65 63 74 69 6E 67 20 73  nd redirecting s
00000050  74 64 65 72 72 20 74 6F - 20 73 74 64 6F 75 74 0A  tderr to stdout.
00000060  23 0A 0A 73 65 74 20 6F - 70 74 69 6F 6E 73 20 3D  #..set options =
00000070  20 22 22 0A 69 66 20 28 - 24 4F 53 54 59 50 45 20   "".if ($OSTYPE 
00000080  3D 3D 20 6C 69 6E 75 78 - 29 20 74 68 65 6E 0A 20  == linux) then. 
00000090  20 20 20 73 65 74 20 6F - 70 74 69 6F 6E 73 20 3D     set options =
000000A0  20 22 2D 63 20 31 22 0A - 65 6E 64 69 66 0A 70 69   "-c 1".endif.pi
000000B0  6E 67 20 24 6F 70 74 69 - 6F 6E 73 20 24 2A 20 3E  ng $options $* >
000000C0  26 21 20 74 65 6D 70 5F - 24 24 2E 6C 6F 67 0A 63  &! 

In [83]:
## TEST 5 - calc AND ITS ALIASES

## ERROR IN CODE 

# $ calc "9+10"
# | Number found where operator expected at (eval 34) line 1, near "args 9"
# |         (Do you need to predeclare args?)
# | Missing argument in printf at /home/aveey/bin/perlcalc.perl line 117, <> line 1.
# | 0.000


Number found where operator expected at (eval 34) line 1, near "args 9"
	(Do you need to predeclare args?)
Missing argument in printf at /home/aveey/bin/perlcalc.perl line 117, <> line 1.
0.000


In [85]:
## ERROR

# $ perl-calc-init "9+10"
# | Missing argument in printf at /home/aveey/bin/perlcalc.perl line 117, <> line 1.
# | 0.000


Missing argument in printf at /home/aveey/bin/perlcalc.perl line 117, <> line 1.
0.000
